In [1]:
# Note: this data is not filtered between 1-100Hz in this notebook! Filtering is done only in RMSE notebook

# Calculating frequency spectrum and power per each frequency band.

import os

#import matplotlib
#matplotlib.use('MACOSX')
#for some reason if I run these 2 lines - it doesnt plot at all any more.

import numpy as np
import mne
import matplotlib.pyplot as plt
from copy import deepcopy
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch

In [2]:
#sample_data_folder = mne.datasets.sample.data_path()
#kath_raw_file2 = "/Users/jenya/Documents/Oldenburg and university/Job Uni Rieger lab/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif"
kath_raw_file = os.path.join('Katharinas_Data','sub_HT05ND16', '210811', 'mikado-1.fif')

#print(kath_raw_file)
#print(kath_raw_file2)                                   
raw = mne.io.read_raw_fif(kath_raw_file)
#raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

#Print info about the data:
#print(raw)
#print(raw.info)
raw

Opening raw data file Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.

/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_27187/2064368720.py:7: RuntimeWarning: This filename (Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(kath_raw_file)


<Raw | mikado-1.fif, 319 x 3390000 (3390.0 s), ~6.6 MB, data not loaded>

In [3]:
# Frequency spectrum using Welch method:
# But I dont know what windows plot_psd is using for frequency separation and also it does not 
# give frequencies and amplitudes as a separate variable, just the plot.

original_raw = raw.copy() #just in case. need to check what changes the data and what doesnt

#frequency spectrum only for gradiometers:
picks_grad = mne.pick_types(raw.info, meg='grad', eeg=False, eog=False, stim=False)
picks_magn = mne.pick_types(raw.info, meg='mag', eeg=False, eog=False, stim=False)

psds_mags, freqs_mags = psd_welch(original_raw, fmin=0, fmax=200, n_jobs=-1, picks=picks_magn, tmin=300, tmax=480, n_fft=1000, n_per_seg=1000)
psds_grads, freqs_grads = psd_welch(original_raw, fmin=0, fmax=200, n_jobs=-1, picks=picks_grad, tmin=300, tmax=480, n_fft=1000, n_per_seg=1000)
# n_per_seg - Length of each Welch segment (windowed with a Hamming window). Defaults to None, which sets n_per_seg equal to n_fft.
# n_fft - The length of FFT used, must be >= n_per_seg (default: 256). The segments will be zero-padded if n_fft > n_per_seg. If n_per_seg 
# is None, n_fft must be <= number of time points in the data.
# These influence the bandwidth.

# ? Understandable explanation of parameters to be put here :)


Effective window size : 1.000 (s)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.8s remaining:    4.6s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.9s remaining:    1.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    3.0s finished


Effective window size : 1.000 (s)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.4s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.8s finished


In [4]:
freqs_mat_mags=np.tile(freqs_mags, [np.shape(psds_mags)[0],1])
freqs_mat_grads=np.tile(freqs_grads, [np.shape(psds_grads)[0],1])

from matplotlib import pyplot as plt

%matplotlib qt
#%matplotlib inline

#Magnetometers:
plt.figure()
plt.plot(freqs_mat_mags.T, np.sqrt(psds_mags.T))
plt.yscale='log'
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (T / Hz)')  #check the units!
plt.title("Welch's periodogram for all magnetometers")
plt.show()

#Freq spectrum peaks we see:
#50, 100, 150 - powerline EU
#6 noise of shielding chambers 
#44 meg noise
#17 - was it the train station near by?
#10 Secret :)
#1hz - highpass filter.
#flat spectrum is white noise process. Has same energy in every frequency (starts around 50Hz or even below)

#Gradiometers:
plt.figure()
plt.plot(freqs_mat_grads.T, np.sqrt(psds_grads.T))
plt.yscale='log'
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (T/m)²/Hz)')  #check the units!
plt.title("Welch's periodogram for all gradiometers")
plt.show()

#Need to find frequencies.. and filter out? 
#Powerline
#Eye moves 
#Blinks
#Cardio: try to autocreate it. Maybe it s small enough to not care?
#Muscle movements 



qt.qpa.drawing: Layer-backing can not be explicitly controlled on 10.14 when built against the 10.14 SDK


In [5]:
#For frequency spectrum do we need to estimate for the entire duration or just per epoch?

#What we did in course:
#Filter the whole data bandpass: 0.3-200 Hz -  frequencies we r inetersted in
#cutting out separate epochs. so he have data as: channel*epoch of particular duration
#calculate frequency spectrum (seems for the entire duration. why did we cut then?)

#__________________
# What we do here:
# Take 1 band, delta or so and calculate area under the curve. 
# Or: take one band and calculate std
# Devide area under curve by num of freq. Should be same as rmse for this band.

# Bands info: https://raphaelvallat.com/bandpower.html
# delta (0.5–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (12–30 Hz), and gamma (30–100 Hz).
# Code adopted from the bands article above:

#THIS CELL CALCULATES ONLY FOR channel#2 (indexed as 1) in grads AND PLOTS THE RESULT:
#Use this cell to check the results of the next cell!

from scipy.integrate import simps

wave_bands=[[0.5, 4], [4, 8], [8, 12], [12, 30], [30, 100]]

for w in wave_bands:

    # Define delta lower and upper limits
    low, high = w

    # Find intersecting values in frequency vector

    # Take just 1 channel for now:
    # idx_delta = np.logical_and(freqs >= low, freqs <= high)

    idx_delta_grads = np.logical_and(freqs_grads >= low, freqs_grads <= high)

    psd_ch1=psds_grads[1, :]

    # Plot the power spectral density and fill the delta area
    plt.figure(figsize=(7, 4))
    plt.plot(freqs_grads, psd_ch1, lw=2, color='k')
    plt.fill_between(freqs_grads, psd_ch1, where=idx_delta_grads, color='skyblue')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power spectral density (uV^2 / Hz)')
    plt.xlim([0, 100])
    plt.ylim([0, psd_ch1.max() * 1.1])
    plt.title("Welch's periodogram")
    plt.show()

    #Area under the curve

    # Frequency resolution
    freq_res = freqs_grads[1] - freqs_grads[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    delta_power = simps(psd_ch1[idx_delta_grads], dx=freq_res)
    print('Absolute power of this band in V^2: ', delta_power)

    #delta_compare=delta_power/len(idx_delta_mags==True) #devide the power by the  number of frequencies in the band

    delta_compare=delta_power/sum(idx_delta_grads)
    print('Power per frequency (to compare with RMSE): ', delta_compare) #HOW TO NAME THIS VALUE?



Absolute power of this band in V^2:  2.8744149414866174e-23
Power per frequency (to compare with RMSE):  7.186037353716543e-24
Absolute power of this band in V^2:  6.342252606541744e-24
Power per frequency (to compare with RMSE):  1.2684505213083487e-24
Absolute power of this band in V^2:  3.3867648919714376e-24
Power per frequency (to compare with RMSE):  6.773529783942875e-25
Absolute power of this band in V^2:  8.085150651251878e-24
Power per frequency (to compare with RMSE):  4.255342448027304e-25
Absolute power of this band in V^2:  8.964478957620845e-24
Power per frequency (to compare with RMSE):  1.262602670087443e-25


In [6]:
# Power calculation for all mags + grads channels separately, 
# saving power + power/freq value in data frames.

from scipy.integrate import simps
import pandas as pd

#Separate mags and grads:
mags = [(chs['ch_name'], i) for i, chs in enumerate(raw.info['chs']) if str(chs['unit']).endswith('UNIT_T)')]
grads = [(chs['ch_name'], i) for i, chs in enumerate(raw.info['chs']) if str(chs['unit']).endswith('UNIT_T_M)')]

# Calculate the band power:
wave_bands=[[0.5, 4], [4, 8], [8, 12], [12, 30], [30, 100]]
#delta (0.5–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (12–30 Hz), and gamma (30–100 Hz) bands

mags_names = [mag[0] for mag in mags]
grads_names = [grad[0] for grad in grads]

dict_mags_power = {}
dict_grads_power = {}

dict_mags_power_freq = {}
dict_grads_power_freq = {}

for w in enumerate(wave_bands): #loop over bands

    low, high = w[1] # Define delta lower and upper limits
    power_per_band = {"mags": [], "grads": []}
    power_per_freq_per_band = {"mags": [], "grads": []}

  #loop over mags, then grads

    idx_delta_m = np.logical_and(freqs_mags >= low, freqs_mags <= high)
    for m in enumerate(psds_mags): 
    #loop over mags channels. psd_ch_m is psd of partigular channel

        ch_name_m=mags_names[m[0]]
        psd_ch_m=np.array(m[1])

        #Area under the curve:
        # Frequency resolution
        freq_res = freqs_mags[1] - freqs_mags[0]  # = 1 / 4 = 0.25

        # Compute the absolute power by approximating the area under the curve
        band_power_m = simps(psd_ch_m[idx_delta_m], dx=freq_res)

        #devide the power by the  number of frequencies in the band
        power_compare_m=band_power_m/sum(idx_delta_m) 

        power_per_band['mags'].append(band_power_m)
        power_per_freq_per_band['mags'].append(power_compare_m)

    #print('mags done')

    idx_delta_g = np.logical_and(freqs_grads >= low, freqs_grads <= high)
    for g in enumerate(psds_grads): 
    #loop over grads channels and their names

        ch_name_g=grads_names[g[0]]
        psd_ch_g=np.array(g[1])

        #Area under the curve:
        # Frequency resolution
        freq_res = freqs_grads[1] - freqs_grads[0]  

        # Compute the absolute power by approximating the area under the curve
        band_power_g = simps(psd_ch_g[idx_delta_g], dx=freq_res)

        #devide the power by the  number of frequencies in the band
        power_compare_g=band_power_g/sum(idx_delta_g) 

        power_per_band['grads'].append(band_power_g)
        power_per_freq_per_band['grads'].append(power_compare_g)

    #print('grads done')

        
    dict_mags_power[w[0]] = power_per_band["mags"]
    dict_grads_power[w[0]] = power_per_band["grads"]

    dict_mags_power_freq[w[0]] = power_per_freq_per_band["mags"]
    dict_grads_power_freq[w[0]] = power_per_freq_per_band["grads"]


# Save power and delta_compare to data frame:
df_power_mags = pd.DataFrame(dict_mags_power, index=mags_names)
df_power_grads = pd.DataFrame(dict_grads_power, index=grads_names)

df_power_freq_mags = pd.DataFrame(dict_mags_power_freq, index=mags_names)
df_power_freq_grads = pd.DataFrame(dict_grads_power_freq, index=grads_names)


In [7]:
# Rename columns and extract to csv:

renamed_df_power_mags = df_power_mags.rename(columns={0: "delta (0.5-4 Hz)", 1: "theta (4-8 Hz)", 2: "alpha (8-12 Hz)", 3: "beta (12-30 Hz)", 4: "gamma (30-100 Hz)"})
renamed_df_power_grads = df_power_grads.rename(columns={0: "delta (0.5-4 Hz)", 1: "theta (4-8 Hz)", 2: "alpha (8-12 Hz)", 3: "beta (12-30 Hz)", 4: "gamma (30-100 Hz)"})

renamed_df_power_freq_mags = df_power_freq_mags.rename(columns={0: "delta (0.5-4 Hz)", 1: "theta (4-8 Hz)", 2: "alpha (8-12 Hz)", 3: "beta (12-30 Hz)", 4: "gamma (30-100 Hz)"})
renamed_df_power_freq_grads = df_power_freq_grads.rename(columns={0: "delta (0.5-4 Hz)", 1: "theta (4-8 Hz)", 2: "alpha (8-12 Hz)", 3: "beta (12-30 Hz)", 4: "gamma (30-100 Hz)"})

renamed_df_power_mags.to_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/abs_power_mags.csv')
renamed_df_power_grads.to_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/abs_power_grads.csv')
renamed_df_power_freq_mags.to_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/power_per_freq_mags.csv')
renamed_df_power_freq_grads.to_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/power_per_freq_grads.csv')

#Look at df:
#df_power_mags
     

In [14]:
#Calculate power per band over all mags and all grads

import statistics 

power_dfs=[df_power_mags, df_power_grads, df_power_freq_mags, df_power_freq_grads]
bands_names=['delta', 'theta', 'alpha', 'beta', 'gamma']
band_title=['Magnetometers. Average absolute power per band:', 'Gradiometers. Average absolute power per band:', 'Magnetometers. Average power/freq per band:', 'Gradiometers. Average power/freq per band:' ]

for d in enumerate(power_dfs):
    print(band_title[d[0]])

    for w in enumerate(bands_names): #loop over bands
        mean_power_per_band = statistics.mean(d[1].loc[:,w[0]])
        print(w[1], mean_power_per_band)
    



Magnetometers. Average absolute power per band:
delta 6.325767081342621e-26
theta 2.8726870011584566e-26
alpha 1.7327560939401702e-26
beta 3.528272128171432e-26
gamma 1.2712285355202972e-26
Gradiometers. Average absolute power per band:
delta 7.930487347659766e-24
theta 3.524096738547444e-24
alpha 3.540721121183862e-24
beta 5.808547134334731e-24
gamma 9.56435910378863e-24
Magnetometers. Average power/freq per band:
delta 1.5814417703356553e-26
theta 5.745374002316913e-27
alpha 3.4655121878803404e-27
beta 1.856985330616543e-27
gamma 1.7904627260849256e-28
Gradiometers. Average power/freq per band:
delta 1.9826218369149414e-24
theta 7.048193477094888e-25
alpha 7.081442242367723e-25
beta 3.05713007070249e-25
gamma 1.3470928315195255e-25


In [ ]:
# Calculate stds per band and compare with value above? Do we need that?



In [ ]:
# Finding peaks?
# 
# https://mne.tools/stable/generated/mne.preprocessing.peak_finder.html?highlight=peak

In [ ]:
# Turn it into a function/module